In [2]:
import pandas as pd
import numpy as np
import gc
from scipy.stats import pearsonr
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_selection import mutual_info_classif

# 读取特征和标签

In [3]:
total_factor = pd.read_pickle('/home/datamake117/data/haris/dataset_1209/total_factor.pkl')
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
0,2020-01-02,000001,0.161906,0.715487,0.093825,2.500486,1.495386,3.687422,1.265125,2.488541,...,10500.0,70.0,1.0,0.100181,0.0,0.010385,0,0,0,1.886999e+07
1,2020-01-02,000002,0.233131,0.554393,-0.052147,0.148151,0.797523,-4.361952,0.735354,0.153694,...,10564.0,67.0,1.0,0.100095,0.0,-0.044313,0,0,0,2.000000e+07
2,2020-01-02,000004,0.129032,0.902306,-0.089413,-3.294483,0.073779,-4.185775,0.243243,-3.187878,...,10579.0,61.0,0.0,0.100044,0.0,-0.034692,0,0,0,2.831250e+04
3,2020-01-02,000005,0.003936,0.962999,0.011201,-2.758275,1.244934,-4.228945,0.402985,-2.640067,...,10605.0,62.0,0.0,0.100649,0.0,-0.008585,0,0,0,1.079170e+05
4,2020-01-02,000006,0.059158,0.808410,-0.066815,0.230044,0.572983,-3.184859,0.476190,0.285741,...,10110.0,62.0,1.0,0.099624,0.0,-0.012239,0,0,0,2.818170e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577802,2025-03-04,688799,0.009111,0.813119,-0.165644,-3.837459,0.087797,-4.701649,0.114286,-3.864892,...,1330.0,125.0,1.0,0.200100,0.0,-0.003425,0,0,0,7.907900e+04
5577803,2025-03-04,688800,0.422390,0.801728,0.260302,1.368099,2.970754,1.809854,1.785714,1.321393,...,1321.0,129.0,1.0,0.200000,0.0,0.019251,0,0,0,2.469798e+06
5577804,2025-03-04,688819,0.182125,0.835346,-0.071127,0.678837,0.440051,2.464627,1.551724,0.885954,...,1506.0,124.0,1.0,0.199860,0.0,0.001270,0,0,0,5.152901e+05
5577805,2025-03-04,688981,0.053828,0.821094,-0.028884,0.118893,0.708258,1.227951,1.085288,0.232366,...,1692.0,115.0,1.0,0.200020,0.0,0.004062,0,0,0,1.856219e+07


In [4]:
temp = total_factor[total_factor['date'] >= pd.to_datetime("2020-07-01")]
del total_factor
total_factor = temp
del temp
gc.collect()
total_factor

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
424421,2020-07-01,000001,0.032755,0.843080,-0.074178,-3.458246,0.447829,-7.749803,0.534989,-3.410864,...,10682.0,70.0,1.0,0.100000,0.0,0.031135,0,0,0,3.718053e+06
424422,2020-07-01,000002,0.086861,0.737642,-0.144779,-0.831296,0.356911,-3.153102,0.800866,-0.782118,...,10746.0,63.0,1.0,0.099923,0.0,0.065511,0,0,0,8.181774e+06
424423,2020-07-01,000004,0.632580,0.284831,-0.511711,-1.308610,0.095793,-4.276766,0.350877,-1.302909,...,10761.0,63.0,0.0,0.100135,0.0,-0.030649,0,0,0,8.880400e+05
424424,2020-07-01,000005,0.000000,0.976718,-0.009933,0.996359,0.529879,-2.759158,0.360000,1.021340,...,10787.0,38.0,0.0,0.100386,0.0,0.003909,0,0,0,1.841000e+04
424425,2020-07-01,000006,0.108898,0.533500,-0.257871,-1.452850,0.265997,2.785404,1.405660,-1.434770,...,10292.0,61.0,1.0,0.100152,0.0,0.099182,0,0,0,1.713309e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577802,2025-03-04,688799,0.009111,0.813119,-0.165644,-3.837459,0.087797,-4.701649,0.114286,-3.864892,...,1330.0,125.0,1.0,0.200100,0.0,-0.003425,0,0,0,7.907900e+04
5577803,2025-03-04,688800,0.422390,0.801728,0.260302,1.368099,2.970754,1.809854,1.785714,1.321393,...,1321.0,129.0,1.0,0.200000,0.0,0.019251,0,0,0,2.469798e+06
5577804,2025-03-04,688819,0.182125,0.835346,-0.071127,0.678837,0.440051,2.464627,1.551724,0.885954,...,1506.0,124.0,1.0,0.199860,0.0,0.001270,0,0,0,5.152901e+05
5577805,2025-03-04,688981,0.053828,0.821094,-0.028884,0.118893,0.708258,1.227951,1.085288,0.232366,...,1692.0,115.0,1.0,0.200020,0.0,0.004062,0,0,0,1.856219e+07


In [5]:
# 只保留2020-07-01至2024-09-30的数据
total_factor = total_factor.fillna(0)
total_factor.head()

,date,Code,0,1,2,3,4,5,6,7,...,2216,2217,2218,2219,2220,label,qcut,weight,qid,amount
424421,2020-07-01,000001,0.032755,0.843080,-0.074178,-3.458246,0.447829,-7.749803,0.534989,-3.410864,...,10682.0,70.0,1.0,0.100000,0.0,0.031135,0,0,0,3718053.0
424422,2020-07-01,000002,0.086861,0.737642,-0.144779,-0.831296,0.356911,-3.153102,0.800866,-0.782118,...,10746.0,63.0,1.0,0.099923,0.0,0.065511,0,0,0,8181773.6
424423,2020-07-01,000004,0.632580,0.284831,-0.511711,-1.308610,0.095793,-4.276766,0.350877,-1.302909,...,10761.0,63.0,0.0,0.100135,0.0,-0.030649,0,0,0,888040.0
424424,2020-07-01,000005,0.000000,0.976718,-0.009933,0.996359,0.529879,-2.759158,0.360000,1.021340,...,10787.0,38.0,0.0,0.100386,0.0,0.003909,0,0,0,18410.0
424425,2020-07-01,000006,0.108898,0.533500,-0.257871,-1.452850,0.265997,2.785404,1.405660,-1.434770,...,10292.0,61.0,1.0,0.100152,0.0,0.099182,0,0,0,1713309.0


# 构造因子筛选辅助数据

In [6]:
factor_columns = [col for col in total_factor.columns if col not in ['date', 'Code', 'label', 'week_group', 'qcut', 'weight', 'qid', 'amount']]

# 计算因子每天的ic，注释掉的部分可以任意筛掉不同收益率分位数的股票
def compute_all_corrs(df, power=1, target_col='label'):
    df = df.dropna()
    data = df[factor_columns].to_numpy()
    target = df[target_col].to_numpy()
    # 用因子的几次方计算ic
    data = data ** power
    # 中心化数据
    target_centered = target - target.mean()
    data_centered = data - data.mean(axis=0)
    # 协方差
    covariances = np.mean(data_centered * target_centered[:, np.newaxis], axis=0)
    # 标准差
    std_target = target.std()
    std_features = data.std(axis=0)
    std_features[std_features==0] = 1
    # 相关系数
    correlations = covariances / (std_target * std_features)

    del df
    del target
    del data
    del target_centered
    del data_centered
    del std_target
    del std_features
    del covariances
    gc.collect()
    
    return pd.Series(correlations, index=factor_columns)

In [7]:
# 按天分组计算相关性
correlation_df1 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=1)).reset_index()
correlation_df2 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=2)).reset_index()
correlation_df3 = total_factor.groupby('date').apply(lambda x: compute_all_corrs(x, power=3)).reset_index()
correlation_df1.set_index('date', inplace=True)
correlation_df2.set_index('date', inplace=True)
correlation_df3.set_index('date', inplace=True)
correlation_df1 = correlation_df1.abs()
correlation_df2 = correlation_df2.abs()
correlation_df3 = correlation_df3.abs()
correlation_df = pd.DataFrame(
    np.maximum.reduce([correlation_df1.values, correlation_df2.values, correlation_df3.values]),  # 逐元素最大值
    columns=correlation_df1.columns, 
    index=correlation_df1.index
    )
correlation_df.to_pickle('/home/datamake117/data/haris/dataset_1209/corr_byday_abs.pkl')
correlation_df

/tmp/ipykernel_2465038/533047600.py:12: RuntimeWarning: invalid value encountered in subtract
  data_centered = data - data.mean(axis=0)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/ipykernel_2465038/533047600.py:12: RuntimeWarning: invalid value encountered in subtract
  data_centered = data - data.mean(axis=0)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/home/datamake117/.local/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/t

,0,1,2,3,4,5,6,7,8,9,...,2211,2212,2213,2214,2215,2216,2217,2218,2219,2220
date,,,,,,,,,,,,,,,,,,,,,
2020-07-01,0.088645,0.106941,0.079099,0.018833,0.034070,0.022846,0.006776,0.016486,0.032768,0.033902,...,0.073248,0.009295,0.003175,0.013635,0.184713,0.199430,0.014499,0.168110,0.037862,0.022904
2020-07-02,0.094737,0.106021,0.068409,0.043837,0.040171,0.068008,0.024757,0.041063,0.039871,0.041128,...,0.030664,0.030534,0.016962,0.069561,0.160583,0.162133,0.025338,0.160265,0.089617,0.023663
2020-07-03,0.046241,0.024116,0.045347,0.045956,0.025120,0.084884,0.031167,0.047984,0.025727,0.025536,...,0.004908,0.048414,0.021558,0.087144,0.242102,0.206780,0.033080,0.220550,0.039345,0.088107
2020-07-06,0.085844,0.128455,0.066288,0.112957,0.042219,0.141431,0.057618,0.112865,0.044517,0.042595,...,0.030635,0.060434,0.010474,0.030412,0.109997,0.018742,0.026932,0.085496,0.064598,0.059533
2020-07-07,0.108295,0.071632,0.050132,0.050239,0.018193,0.041972,0.047500,0.048509,0.021231,0.018088,...,0.092688,0.049313,0.001939,0.018688,0.060075,0.085980,0.008954,0.041254,0.010750,0.030132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-25,0.155472,0.150036,0.047820,0.060708,0.033730,0.041631,0.050314,0.059054,0.048806,0.037508,...,0.042326,0.078394,0.003877,0.010479,0.034563,0.044445,0.031242,0.014362,0.001453,0.015566
2025-02-26,0.061938,0.086562,0.051787,0.023703,0.016325,0.018457,0.015107,0.020818,0.016211,0.017852,...,0.118036,0.082042,0.001314,0.017342,0.119817,0.067560,0.033304,0.031078,0.104531,0.007773
2025-02-28,0.165372,0.219750,0.115041,0.048022,0.030448,0.102321,0.054413,0.048919,0.033329,0.029767,...,0.096338,0.072214,0.007495,0.008040,0.010386,0.034696,0.031764,0.000776,0.002143,0.034673
